In [1]:
import numpy as np
import copy
import sys

with open("input.txt", 'r') as f:
    input = f.read()

# part 2 should give in total 285
input1 = """###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############"""

def print_map(racemap):
    print("\n".join(["".join(row) for row in racemap]))

racemap = np.array([list(row) for row in input.split("\n")])
print_map(racemap)
START = tuple(np.argwhere(racemap == "S")[0].tolist())
END = tuple(np.argwhere(racemap == "E")[0].tolist())
START, END

#############################################################################################################################################
#...#...#...........#...#...#...#...#.....#...###...#...###.........#...#...#.......#...#...#...#...#...#...###...#...###...#...###.........#
#.#.#.#.#.#########.#.#.#.#.#.#.#.#.#.###.#.#.###.#.#.#.###.#######.#.#.#.#.#.#####.#.#.#.#.#.#.#.#.#.#.#.#.###.#.#.#.###.#.#.#.###.#######.#
#.#...#.#...#.....#...#...#...#.#.#.#...#...#...#.#...#...#.......#...#...#.#.....#.#.#...#.#.#.#.#.#.#...#.#...#.#.#.....#.#.#.#...#...#...#
#.#####.###.#.###.#############.#.#.###.#######.#.#######.#######.#########.#####.#.#.#####.#.#.#.#.#.#####.#.###.#.#######.#.#.#.###.#.#.###
#.#.....###...###.......#.......#.#.....#.......#.......#...#.....#...#.....#...#.#.#.....#...#.#.#.#.#.....#.#...#.#.......#.#.#.#...#...###
#.#.###################.#.#######.#######.#############.###.#.#####.#.#.#####.#.#.#.#####.#####.#.#.#.#.#####.#.###.#.#######.#.#.#.#########
#.#.#.

((75, 17), (93, 7))

In [2]:
a= []
a = a.extend([80])
a

sys.setrecursionlimit(int(1e4))

In [3]:
MEMORY = {}


DIRECTIONS = [(-1,0), # up
              (0,1),  # right
              (1,0),
              (0,-1)]

DIR_2_STR = {
    (-1,0): "^",
    (0,1): ">",
    (1,0): "v",
    (0,-1): "<",
}

def in_map(pos, racemap, conservative = False):
    if not conservative:
        return pos[0] in range(racemap.shape[0]) and pos[1] in range(racemap.shape[1])
    else:
        return pos[0] in range(1, racemap.shape[0]-1) and pos[1] in range(1, racemap.shape[1]-1)



def explore(racemap: np.ndarray, prev_pos: tuple[int,int], pos: tuple[int,int], end: tuple[int,int], cheat_left: bool, picoseconds:int) -> list[int]:
    global MEMORY
    # print(pos)x
    if pos == end:
        return [picoseconds]
    if MEMORY.get((pos[0],pos[1],cheat_left, picoseconds, prev_pos[0], prev_pos[1]), -1) != -1:
        return MEMORY[(pos[0], pos[1], cheat_left, picoseconds, prev_pos[0], prev_pos[1])]
    all_picoseconds = []
    for dir in DIRECTIONS:
        new_pos = pos[0]+dir[0], pos[1]+dir[1]
        if racemap[new_pos] == "*":
            continue
        cheat_pos = pos[0]+2*dir[0], pos[1]+2*dir[1]
        if racemap[new_pos] == "#" and (not cheat_left or not in_map(cheat_pos, racemap) or racemap[cheat_pos] in ["#", "*"]):
            # we cannot cheat
            continue
        
        racemap_modified = copy.copy(racemap)
        racemap_modified[pos] = "*"
        if racemap[new_pos] == "#":
            # we can cheat
            ps = explore(racemap_modified, prev_pos=pos, pos=cheat_pos, end=end, cheat_left=False, picoseconds=picoseconds+2)
            all_picoseconds += ps
        elif racemap[new_pos] in [".", "E"]:
            ps = explore(racemap_modified, prev_pos=pos, pos=new_pos, end=end, cheat_left=cheat_left, picoseconds=picoseconds+1)
            all_picoseconds += ps

    MEMORY[(pos[0], pos[1], cheat_left, picoseconds, prev_pos[0], prev_pos[1])] = all_picoseconds
    return all_picoseconds

possible_times = explore(racemap, prev_pos=START, pos=START, end=END, cheat_left=True, picoseconds=0)
possible_deltas = [max(possible_times)-t for t in possible_times]
print(len([d for d in possible_deltas if d >= 100]))
        

1343


### Part 2

In [4]:
def simple_solve(racemap, start):
    # get the baseline (worst) solution
    racemap_modified = copy.copy(racemap)
    worst_path = [(start)]
    pos = start
    while True:
        if racemap_modified[pos] == "E":
            break
        for dir in DIRECTIONS:
            new_pos = pos[0]+dir[0], pos[1]+dir[1]
            if racemap_modified[new_pos] not in ["#", "*"]:
                worst_path.append(new_pos)
                racemap_modified[pos] = "*"
                pos = new_pos
                break
    return worst_path

worst_path = simple_solve(racemap, START)


In [5]:
racemap_int = None
racemap_modified = None

def manhattan_distance(start, end):
    return abs(start[0]-end[0]) + abs(start[1]-end[1])


def outer_loop(worst_path, improvement_limit=50, budget=20):
    global racemap_modified, racemap_int
    total = 0
    for i, start_pos in enumerate(worst_path[:-improvement_limit]):
        print(i, len(worst_path))
        for j, end_pos in enumerate(worst_path[i+improvement_limit:]):
            delta = manhattan_distance(start_pos, end_pos)
            if delta > budget:
                continue
           
            savings = (j+i+improvement_limit) - i - (delta)
            if savings >= improvement_limit:
                total +=1
    return total


    

outer_loop(worst_path, improvement_limit=100, budget=20)


0 9377
1 9377
2 9377
3 9377
4 9377
5 9377
6 9377
7 9377
8 9377
9 9377
10 9377
11 9377
12 9377
13 9377
14 9377
15 9377
16 9377
17 9377
18 9377
19 9377
20 9377
21 9377
22 9377
23 9377
24 9377
25 9377
26 9377
27 9377
28 9377
29 9377
30 9377
31 9377
32 9377
33 9377
34 9377
35 9377
36 9377
37 9377
38 9377
39 9377
40 9377
41 9377
42 9377
43 9377
44 9377
45 9377
46 9377
47 9377
48 9377
49 9377
50 9377
51 9377
52 9377
53 9377
54 9377
55 9377
56 9377
57 9377
58 9377
59 9377
60 9377
61 9377
62 9377
63 9377
64 9377
65 9377
66 9377
67 9377
68 9377
69 9377
70 9377
71 9377
72 9377
73 9377
74 9377
75 9377
76 9377
77 9377
78 9377
79 9377
80 9377
81 9377
82 9377
83 9377
84 9377
85 9377
86 9377
87 9377
88 9377
89 9377
90 9377
91 9377
92 9377
93 9377
94 9377
95 9377
96 9377
97 9377
98 9377
99 9377
100 9377
101 9377
102 9377
103 9377
104 9377
105 9377
106 9377
107 9377
108 9377
109 9377
110 9377
111 9377
112 9377
113 9377
114 9377
115 9377
116 9377
117 9377
118 9377
119 9377
120 9377
121 9377
122 9377
123

982891

In [6]:
# To achieve the above solution, I took inspiration of:
# https://github.com/alan-turing-institute/advent-of-code-2024/blob/main/day-20/python_adun/day20.ipynb

def get_neighbours(maze, pos):
    neighbours = []
    for dir in DIRECTIONS:
        new_pos =  pos[0] + dir[0], pos[1] + dir[1]

        if maze[new_pos[0]][new_pos[1]] != '#':
            neighbours.append(new_pos)

    return neighbours

def bfs(maze, start):
    distances = {start: 0}
    queue = [(0, start)]
    path = []
    
    while queue:
        dist, pos = queue.pop(0)
        
        path.append(pos)
            
        for next_pos in get_neighbours(maze, pos):
            if next_pos not in distances:
                distances[next_pos] = dist + 1
                queue.append((dist + 1, next_pos))
                
    return distances, path

def find_tricks_part2(path, dists, max_delta):
    path_rev = path[::-1]
    num_tricks = 0
    for i in range(len(path_rev)):
        for j in range(i+1, len(path_rev)):
            pos1, pos2 = path_rev[i], path_rev[j]
            delta = abs(pos1[0]-pos2[0]) + abs(pos1[1]-pos2[1])
            if delta <= max_delta:
                new_cost = dists[pos1] - dists[pos2]-delta
                if new_cost >= 100:
                    num_tricks +=1

    return num_tricks

dists, path = bfs(racemap, START)
find_tricks_part2(path, dists, 20)


982891